## Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
#import pyarrow.feather as feather
from sklearn.model_selection import train_test_split 
from tensorflow.keras.callbacks import ReduceLROnPlateau
#from tensorflow.keras.layers.experimental import preprocessing


# W & B test

In [ ]:
# import wandb
# from wandb.keras import WandbCallback
# !wandb login


## Prepare the data

In [ ]:
# fluxData_df = feather.read_feather('data/fluxData.feather')
# # zernikeData_df = feather.read_feather('data/zernikeData.feather')
# fluxData_df


## import data as h5

In [ ]:
fluxData_df =pd.read_hdf("data/fluxData_df.h5", key="fluxData_df")

zernikeData_df =pd.read_hdf("data/zernikeData_df.h5", key="zernikeData_df")

In [ ]:
# Prepare the training dataset
X = fluxData_df
y = zernikeData_df
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42)


## Normalize data
as shown in https://keras.io/guides/preprocessing_layers/#normalizing-numerical-features 

In [ ]:

x_train_norm = (x_train - x_train.mean(axis=0)) / x_train.std(axis=0)
x_train_norm

# Build Model

In [10]:
from tensorflow import keras
AoModel = keras.Sequential([
     keras.layers.InputLayer(19, name="digits"),
     keras.layers.Dense(2000, activation="relu"),
     keras.layers.BatchNormalization(),
     keras.layers.Dense(1050, activation="relu"),
     keras.layers.BatchNormalization(),
     keras.layers.Dense(100, activation="relu"),
     keras.layers.BatchNormalization(),
     keras.layers.Dense(9, activation="linear", name="predictions"),

])
AoModel.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 2000)              40000     
_________________________________________________________________
batch_normalization_1 (Batch (None, 2000)              8000      
_________________________________________________________________
dense_7 (Dense)              (None, 1050)              2101050   
_________________________________________________________________
batch_normalization_2 (Batch (None, 1050)              4200      
_________________________________________________________________
dense_8 (Dense)              (None, 100)               105100    
_________________________________________________________________
batch_normalization_3 (Batch (None, 100)               400       
_________________________________________________________________
predictions (Dense)          (None, 9)                

## Compile and run model 

In [11]:
batch_size = 128
epochs = 200

opt = keras.optimizers.Adam(learning_rate=0.001)
AoModel.compile(loss= keras.losses.MeanSquaredError(),  optimizer=opt, metrics= [tf.keras.metrics.RootMeanSquaredError()])
AoModel.fit(x_train_norm, y_train, batch_size=batch_size, epochs=epochs,validation_split = 0.2, verbose = 2)

#AoModel.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[WandbCallback()])

Train on 42292 samples, validate on 10574 samples
Epoch 1/200
42292/42292 - 3s - loss: 0.0480 - root_mean_squared_error: 0.2190 - val_loss: 0.0241 - val_root_mean_squared_error: 0.1552
Epoch 2/200
42292/42292 - 2s - loss: 0.0069 - root_mean_squared_error: 0.0831 - val_loss: 0.0050 - val_root_mean_squared_error: 0.0710
Epoch 3/200
42292/42292 - 2s - loss: 0.0048 - root_mean_squared_error: 0.0693 - val_loss: 0.0040 - val_root_mean_squared_error: 0.0635
Epoch 4/200
42292/42292 - 2s - loss: 0.0038 - root_mean_squared_error: 0.0620 - val_loss: 0.0027 - val_root_mean_squared_error: 0.0521
Epoch 5/200
42292/42292 - 2s - loss: 0.0033 - root_mean_squared_error: 0.0576 - val_loss: 0.0025 - val_root_mean_squared_error: 0.0503
Epoch 6/200
42292/42292 - 2s - loss: 0.0029 - root_mean_squared_error: 0.0539 - val_loss: 0.0026 - val_root_mean_squared_error: 0.0509
Epoch 7/200
42292/42292 - 2s - loss: 0.0026 - root_mean_squared_error: 0.0513 - val_loss: 0.0019 - val_root_mean_squared_error: 0.0430
Epoch